In [1]:
##YAPILABİLECEKLER

#Missing valueların bulunduğu kullanıcıların missing value olmayan başka satırları varsa
#missing value satırları droplanabilir. (Amaç test setindeki tüm satırları tahmin etmek
#değil, her bir unique kullanıcı için cinsiyet tahmin etmek) (OLMADI)

#Kategorik değişkenler, o kategoriden kaç tane işlem yapılmış tarzında yeni değişkenlere
#dönüştürülüp kategorik değişkenler droplanabilir.

#---------------------

# WE CAN USE TIMESTAMP AND ACTION VALUES TO IMPUTE MISSING PRODUCT-RELATED VALUES, then IMPUTE sellingprice with non-missing
# columns. Lastly, we can impute product_gender_column using all the other values.

In [2]:
##########

In [3]:
# NOT: TEST DATASINDAKİ MISSING VALUELARIN NASIL ELE ALINMASI GEREKTIGINI COZ.

# COMBINE THE TRAIN AND TEST, RARE ENCODE, THEN SPLIT

In [4]:
###################

In [5]:
# YAPILMASI GEREKENLER:
# 1. Missing valueları ne yapcağına karar ver.
# 2. Datadaki kategorikleri dummy variable'a çevir.

In [6]:
#################

In [7]:
import pandas as pd
import numpy as np
import missingno as msno
from datetime import datetime

from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
df = train.copy()
df_test = test.copy()
df.drop_duplicates(inplace=True) # Train set has a lot of duplicates. Since timestamp with split seconds precision can't
# be same for different users and different actions, we drop all the duplicates.

In [98]:
test_ids = test['unique_id']

In [10]:
len(df)

2077356

In [11]:
df['unique_id'].isin(df_test['unique_id'].unique()).any() # Train and Test sets have different unique_id values. (Checked)

False

In [12]:
df.head()# unique_id = customer_id

,time_stamp,contentid,user_action,sellingprice,product_name,brand_id,brand_name,businessunit,product_gender,category_id,Level1_Category_Id,Level1_Category_Name,Level2_Category_Id,Level2_Category_Name,Level3_Category_Id,Level3_Category_Name,gender,unique_id,type
0,2020-12-02T22:26:14.023Z,39918893.0,favorite,3099.0,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir...,8511.0,Electrolux,Beyaz Eşya,Unisex,1272.0,1071.0,Elektronik,1212.0,Beyaz Eşya,1272.0,Çamaşır Makinesi,F,425,train
1,2020-12-08T23:15:04.603Z,3558544.0,favorite,3079.0,WW90J5475FW A+++ 1400 Devir 9 kg Çamaşır Makinesi,3228.0,Samsung,Beyaz Eşya,NaN,1272.0,1071.0,Elektronik,1212.0,Beyaz Eşya,1272.0,Çamaşır Makinesi,F,425,train
2,2020-12-05T16:19:01.157Z,31292729.0,favorite,3999.0,KM 9711 A++ 9 kg Çamaşır Kurutma Makinesi,10989.0,Vestel,Beyaz Eşya,Unisex,1276.0,1071.0,Elektronik,1212.0,Beyaz Eşya,1276.0,Kurutma Makinesi,F,425,train
3,2020-12-05T16:28:00Z,6363103.0,visit,2544.0,CMI 9710 A+++ 1000 Devir 9 kg Çamaşır Makinesi,10989.0,Vestel,Beyaz Eşya,NaN,1272.0,1071.0,Elektronik,1212.0,Beyaz Eşya,1272.0,Çamaşır Makinesi,F,425,train
4,2020-12-02T22:26:59Z,39918893.0,visit,3099.0,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir...,8511.0,Electrolux,Beyaz Eşya,Unisex,1272.0,1071.0,Elektronik,1212.0,Beyaz Eşya,1272.0,Çamaşır Makinesi,F,425,train


In [13]:
a = df.groupby('brand_id')['brand_name'].nunique() # 4 tane id 1den fazla brand_name ile eşleşmekte.
a[a>1]

brand_id
34614.0     2
110447.0    2
966197.0    2
998446.0    2
Name: brand_name, dtype: int64

In [14]:
a = df.groupby('brand_name')['brand_id'].nunique() # Brandname başına düşen brand_id sayısı 1.
a[a>1]

Series([], Name: brand_id, dtype: int64)

In [15]:
df['brand_name'].nunique() #33332
df['brand_id'].nunique() #33328

33328

In [16]:
# Brandname değişkeni daha eşsiz olduğu için brand_id değişkeni droplanabilir.

In [17]:
(df['brand_name'].isnull() == df['brand_id'].isnull()).all() # brand_name ve brand_id NA değerleri
# aynı rowlarda.

True

In [18]:
len(df['contentid'].value_counts()) # Çok fazla sayıda kategorisi olan bir değişken. Her ürünün ayrı
# id'si olduğu için bu normal. Bu kadar çok kategori ile iyi sonuç almak mümkün olmadığı için ve 
# mümkün olsa bile genelleme amacına hizmet etmeyeceği için bu feature datadan çıkarılmalıdır.

482796

In [19]:
a = df.groupby('Level1_Category_Id')['Level1_Category_Name'].nunique()
a[a>1] # Her bir Category ID'ye 1 Category Name düşüyor. Dolayısıyla Level1_Category_Name
# çıkarılabilir.

Series([], Name: Level1_Category_Name, dtype: int64)

In [20]:
a = df.groupby('Level2_Category_Id')['Level2_Category_Name'].nunique()
a[a>1] # Her bir Category ID'ye 1 Category Name düşüyor. Dolayısıyla Level2_Category_Name
# çıkarılabilir.

Series([], Name: Level2_Category_Name, dtype: int64)

In [21]:
a = df.groupby('Level3_Category_Id')['Level3_Category_Name'].nunique()
a[a>1] # Her bir Category ID'ye 1 Category Name düşüyor. Dolayısıyla Level3_Category_Name
# çıkarılabilir.

Series([], Name: Level3_Category_Name, dtype: int64)

In [22]:
df[['category_id', 'Level3_Category_Id']] # category_id ve Level3_Category_Id aynı şeyler gibi
# gözükmekte.

,category_id,Level3_Category_Id
0,1272.0,1272.0
1,1272.0,1272.0
2,1276.0,1276.0
3,1272.0,1272.0
4,1272.0,1272.0
...,...,...
5493263,864.0,864.0
5493264,1186.0,1186.0
5493265,864.0,864.0
5493266,975.0,975.0


In [23]:
a = df.groupby('Level3_Category_Id')['category_id'].nunique()
a[a>1] # Level3_Category_Id başına düşen category_id sayısının birden fazla olduğu durumlar bulunmakta.

Level3_Category_Id
397.0      6
399.0      7
400.0      6
401.0      6
469.0      2
          ..
4540.0    13
4569.0    11
4588.0     2
4589.0    39
4591.0    11
Name: category_id, Length: 166, dtype: int64

In [24]:
a = df.groupby('category_id')['Level3_Category_Id'].nunique()
a[a>1] # category id başına düşen Level_3_Category_Id sayısı 1.

Series([], Name: Level3_Category_Id, dtype: int64)

In [25]:
df['category_id'].nunique() #2240
df['Level3_Category_Id'].nunique() #739

739

In [26]:
(df['category_id'].isna() == df['Level3_Category_Id'].isna()).all() # NA oldukları rowlar aynı.
# Dolayısıyla category_id değeri daha eşsiz olduğu için ve aynı şeylere karşılık geliyor gibi
# gözüktükleri için (hatta category_id daha detaylı) Level3_Category_Id değeri droplanabilir.

True

In [27]:
# Drop the columns that we decided to drop before.
# Type column is not necessary at this point, therefore we can drop it as well.

df = train.drop(['contentid', 'Level1_Category_Name', 'Level2_Category_Name', 'Level3_Category_Name',
        'Level3_Category_Id', 'brand_id', 'type'], axis=1)
df.drop_duplicates(inplace=True)
df_test = test.drop(['contentid', 'Level1_Category_Name', 'Level2_Category_Name', 'Level3_Category_Name',
        'Level3_Category_Id', 'brand_id', 'type'], axis=1)
df_test.drop('gender', inplace=True, axis=1) # Gender column is dropped because it is the target feature and full of NA's.

In [28]:
df.head()

,time_stamp,user_action,sellingprice,product_name,brand_name,businessunit,product_gender,category_id,Level1_Category_Id,Level2_Category_Id,gender,unique_id
0,2020-12-02T22:26:14.023Z,favorite,3099.0,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir...,Electrolux,Beyaz Eşya,Unisex,1272.0,1071.0,1212.0,F,425
1,2020-12-08T23:15:04.603Z,favorite,3079.0,WW90J5475FW A+++ 1400 Devir 9 kg Çamaşır Makinesi,Samsung,Beyaz Eşya,NaN,1272.0,1071.0,1212.0,F,425
2,2020-12-05T16:19:01.157Z,favorite,3999.0,KM 9711 A++ 9 kg Çamaşır Kurutma Makinesi,Vestel,Beyaz Eşya,Unisex,1276.0,1071.0,1212.0,F,425
3,2020-12-05T16:28:00Z,visit,2544.0,CMI 9710 A+++ 1000 Devir 9 kg Çamaşır Makinesi,Vestel,Beyaz Eşya,NaN,1272.0,1071.0,1212.0,F,425
4,2020-12-02T22:26:59Z,visit,3099.0,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir...,Electrolux,Beyaz Eşya,Unisex,1272.0,1071.0,1212.0,F,425


In [29]:
df['user_action'].value_counts() 

visit       1027149
search       771374
favorite     156204
basket        99102
order         23299
Name: user_action, dtype: int64

In [30]:
df.isnull().any()

time_stamp            False
user_action           False
sellingprice           True
product_name           True
brand_name             True
businessunit           True
product_gender         True
category_id            True
Level1_Category_Id     True
Level2_Category_Id     True
gender                False
unique_id             False
dtype: bool

In [31]:
len(df) #2077128

2077128

In [32]:
df.isnull().sum() # There are a lot of missing product_gender values. It is missing in roughly 12% of the data. We can't
# drop its missing values, because it has missing values in the test data as well. We can impute it, or we can drop
# the column. (2 Options: Impute gender and/or Drop Column). Another option is to drop all the missing values in both data
# sets and hope that we don't lose any unique ID. (Extra Options: Drop missing values in both sets if possible)

time_stamp                 0
user_action                0
sellingprice           32012
product_name            2184
brand_name              2184
businessunit            2184
product_gender        234567
category_id             2184
Level1_Category_Id      2184
Level2_Category_Id      2184
gender                     0
unique_id                  0
dtype: int64

In [33]:
df_test.isnull().sum()

time_stamp                 0
user_action                0
sellingprice           41368
product_name            2177
brand_name              2177
businessunit            2177
product_gender        275570
category_id             2177
Level1_Category_Id      2177
Level2_Category_Id      2177
unique_id                  0
dtype: int64

In [34]:
s = df_test.notnull().all(axis=1)

In [35]:
notnull = s[s].index

In [36]:
len(df_test.unique_id.unique())

2380

In [37]:
len(df_test.loc[notnull,:].unique_id.unique()) # There are 2380-2331 = 49 unique_id values that has at least 1 missing value
# in their features, therefore we can't directly drop rows with missing values and apply majority voting.

2331

In [38]:
df[df['product_name'].isnull()].notnull().any() #product_name, brand_name, business_unit, category_id, Level1_Category_Id
# and Level2_Category_Id missing values are in the same rows. There are 2184 of these rows.

time_stamp             True
user_action            True
sellingprice           True
product_name          False
brand_name            False
businessunit          False
product_gender        False
category_id           False
Level1_Category_Id    False
Level2_Category_Id    False
gender                 True
unique_id              True
dtype: bool

In [39]:
df[~(df['product_name'].notnull())]

,time_stamp,user_action,sellingprice,product_name,brand_name,businessunit,product_gender,category_id,Level1_Category_Id,Level2_Category_Id,gender,unique_id
10182,2020-11-27T06:40:58Z,visit,65.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,1983
17988,2020-10-21T21:52:56Z,visit,563.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,737
17989,2020-10-21T21:51:38Z,visit,563.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,737
18724,2020-10-22T19:36:40Z,visit,563.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,737
20323,2020-10-21T21:52:00Z,visit,563.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,737
...,...,...,...,...,...,...,...,...,...,...,...,...
5483497,2020-11-28T13:36:52Z,search,559.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,5457
5484856,2020-11-27T23:51:26Z,visit,99.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,3071
5487463,2020-11-29T10:16:38Z,basket,109.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,3354
5487465,2020-11-29T10:16:25Z,visit,109.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,3354


In [40]:
df[~(df['product_name'].notnull())]['user_action'].value_counts()

visit       1100
search       826
basket       120
favorite     102
order         36
Name: user_action, dtype: int64

In [41]:
df[(df['product_name'].notnull())]['user_action'].value_counts() # Missing values are not due to the nature of any action.

visit       1026049
search       770548
favorite     156102
basket        98982
order         23263
Name: user_action, dtype: int64

In [42]:
df[~(df['product_name'].notnull())]['sellingprice'].isnull().any() # True
df[~(df['product_name'].notnull())]['product_gender'].isnull().any() # True
# Both sellingprice and product_gender missing values overlap with product-related columns' missing values at some rows.
# Therefore, we can't use these values to impute missing values of each other.

True

In [43]:
# WE CAN USE TIMESTAMP AND ACTION VALUES TO IMPUTE MISSING PRODUCT-RELATED VALUES, then IMPUTE sellingprice with non-missing
# columns. Lastly, we can impute product_gender_column using all the other values.

In [44]:
#df = df[df['product_name'].notnull()]
#df.isnull().any()
# Removes product-related column NA values, doesn't work for this dataset (See top of the script)

In [45]:
# First, lets try dropping the missing values 

# df.dropna(inplace=True)
# df_test.dropna(inplace=True) # ( This can't be done because we can't drop test data rows.)

In [46]:
# Convert TZ timestamp to datetime64 object

df['timestamp_dt'] = df.apply(lambda x: datetime.fromisoformat(x['time_stamp'].strip('Z')), axis=1)
df_test['timestamp_dt'] = df_test.apply(lambda x: datetime.fromisoformat(x['time_stamp'].strip('Z')), axis=1)

df.drop('time_stamp', axis=1, inplace=True)
df_test.drop('time_stamp', axis=1, inplace=True)

In [47]:
df['timestamp_dt'].min()  # 2020-10-14
df['timestamp_dt'].max()  # 2020-12-09

df_test['timestamp_dt'].min()  # 2020-10-14
df_test['timestamp_dt'].max()  # 2020-12-09
# Data is collected between October 14 and December 9, 2020

Timestamp('2020-12-09 13:29:58')

In [48]:
# I will be creating new variables month, day, day_name hour and minute. I believe that minute and day will not be 
# significant but we can decide it later.

df['weekday'] = df['timestamp_dt'].apply(lambda x: x.weekday())
#df['day'] = df['timestamp_dt'].apply(lambda x: x.day)    - Computationally expensive, creates extra 30 rows.
df['month'] = df['timestamp_dt'].apply(lambda x: x.month)
df['hour'] = df['timestamp_dt'].apply(lambda x: x.hour)
#df['minute'] = df['timestamp_dt'].apply(lambda x: x.minute) - Computationally expensive, probably unimportant.

df.drop(['timestamp_dt', 'unique_id'], axis=1, inplace=True) # Not needed anymore

df_test['weekday'] = df_test['timestamp_dt'].apply(lambda x: x.weekday())
#df_test['day'] = df_test['timestamp_dt'].apply(lambda x: x.day)
df_test['month'] = df_test['timestamp_dt'].apply(lambda x: x.month)
df_test['hour'] = df_test['timestamp_dt'].apply(lambda x: x.hour)
#df_test['minute'] = df_test['timestamp_dt'].apply(lambda x: x.minute)

df_test.drop(['timestamp_dt', 'unique_id'], axis=1, inplace=True) # Not needed anymore

In [49]:
df.head()

,user_action,sellingprice,product_name,brand_name,businessunit,product_gender,category_id,Level1_Category_Id,Level2_Category_Id,gender,weekday,month,hour
0,favorite,3099.0,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir...,Electrolux,Beyaz Eşya,Unisex,1272.0,1071.0,1212.0,F,2,12,22
1,favorite,3079.0,WW90J5475FW A+++ 1400 Devir 9 kg Çamaşır Makinesi,Samsung,Beyaz Eşya,NaN,1272.0,1071.0,1212.0,F,1,12,23
2,favorite,3999.0,KM 9711 A++ 9 kg Çamaşır Kurutma Makinesi,Vestel,Beyaz Eşya,Unisex,1276.0,1071.0,1212.0,F,5,12,16
3,visit,2544.0,CMI 9710 A+++ 1000 Devir 9 kg Çamaşır Makinesi,Vestel,Beyaz Eşya,NaN,1272.0,1071.0,1212.0,F,5,12,16
4,visit,3099.0,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir...,Electrolux,Beyaz Eşya,Unisex,1272.0,1071.0,1212.0,F,2,12,22


In [50]:
df['category_id'] = df['category_id'].astype('object')
df['Level1_Category_Id'] = df['Level1_Category_Id'].astype('object')
df['Level2_Category_Id'] = df['Level2_Category_Id'].astype('object')
df['weekday'] = df['weekday'].astype('object')
df['month'] = df['month'].astype('object')
df['hour'] = df['hour'].astype('object')

sel_price_tr = df['sellingprice']
sel_price_te = df_test['sellingprice']

df_gender = df['gender']

df = df.drop(['sellingprice', 'gender'], axis=1)
df_test = df_test.drop('sellingprice', axis=1)


df.dtypes

user_action           object
product_name          object
brand_name            object
businessunit          object
product_gender        object
category_id           object
Level1_Category_Id    object
Level2_Category_Id    object
weekday               object
month                 object
hour                  object
dtype: object

In [51]:
# Some references

# https://stats.stackexchange.com/questions/440566/classification-model-with-multiple-records-per-user?rq=1
# https://lightgbm.readthedocs.io/en/latest/Advanced-Topics.html
# https://catboost.ai/en/docs/concepts/python-quickstart

In [52]:
# Need to do Rare Encoding before Label Encoding

In [53]:
a = df.nunique()
b = df_test.nunique()

In [54]:
print("Unique Values".rjust(34))
print("-------------".rjust(34))
print("train".rjust(26), "test".rjust(6))
print("-------------".rjust(34))
for i in range(len(a)):
    print(a.index[i].ljust(20), str(a[i]).ljust(6), b[i])

                     Unique Values
                     -------------
                     train   test
                     -------------
user_action          5      5
product_name         441470 239085
brand_name           33332  23943
businessunit         83     83
product_gender       3      3
category_id          2240   2022
Level1_Category_Id   10     10
Level2_Category_Id   95     94
weekday              7      7
month                3      3
hour                 24     24


In [55]:
# Some categorical features have different unique values in train and test. Therefore, we should label the ones that
# are not in the intersection as "Others".

In [56]:
product_name_outliers = set(df['product_name'].dropna()).union(set(df_test['product_name'].dropna()))-\
set(df['product_name'].dropna()).intersection(set(df_test['product_name'].dropna()))
len(product_name_outliers)
# Product name has around 427000 different values. This category is probably useless, considering that only
# around 250000 of the values are common in both data sets.

427659

In [57]:
brand_name_outliers = set(df['brand_name'].dropna()).union(set(df_test['brand_name'].dropna()))-\
set(df['brand_name'].dropna()).intersection(set(df_test['brand_name'].dropna()))
len(brand_name_outliers)
# brand_name_outliers have 17627 different values. This also looks like it will not be useful. We will see.

17627

In [58]:
category_id_outliers = set(df['category_id'].dropna()).union(set(df_test['category_id'].dropna()))-\
set(df['category_id'].dropna()).intersection(set(df_test['category_id'].dropna()))
len(category_id_outliers)
# category_id_outliers have different values

376

In [59]:
level2_outliers = set(df['Level2_Category_Id'].dropna()).union(set(df_test['Level2_Category_Id'].dropna()))-\
set(df['Level2_Category_Id'].dropna()).intersection(set(df_test['Level2_Category_Id'].dropna()))
level2_outliers # Level2 has different levels

{2901.0, 4530.0, 4572.0}

In [60]:
set(df['Level1_Category_Id'].dropna()).union(set(df_test['Level1_Category_Id'].dropna()))-\
set(df['Level1_Category_Id'].dropna()).intersection(set(df_test['Level1_Category_Id'].dropna()))
# Level1 is fine

set()

In [61]:
set(df['businessunit'].dropna()).union(set(df_test['businessunit'].dropna()))-\
set(df['businessunit'].dropna()).intersection(set(df_test['businessunit'].dropna()))
# businessunit is fine

set()

In [62]:
df['type'] = 'train'
df_test['type'] = 'test'
combined_df = pd.concat([df, df_test])

In [63]:
# Replace non-common levels with "Other"

combined_df['product_name'] = combined_df['product_name'].apply(lambda x: "Other" if x in product_name_outliers else x)
combined_df['brand_name'] = combined_df['brand_name'].apply(lambda x: "Other" if x in brand_name_outliers else x)
combined_df['category_id'] = combined_df['category_id'].apply(lambda x: "Other" if x in category_id_outliers else x)
combined_df['Level2_Category_Id'] = combined_df['Level2_Category_Id'].\
apply(lambda x: "Other" if x in level2_outliers else x)

In [64]:
df_withother = combined_df[combined_df['type']=='train']
df_test_withother = combined_df[combined_df['type']=='test']

In [65]:
a = df_withother.nunique()
b = df_test_withother.nunique()

In [66]:
print("Unique Values".rjust(34))
print("-------------".rjust(34))
print("train".rjust(26), "test".rjust(6))
print("-------------".rjust(34))
for i in range(len(a)):
    print(a.index[i].ljust(20), str(a[i]).ljust(6), b[i])

                     Unique Values
                     -------------
                     train   test
                     -------------
user_action          5      5
product_name         126449 126449
brand_name           19825  19825
businessunit         83     83
product_gender       3      3
category_id          1944   1944
Level1_Category_Id   10     10
Level2_Category_Id   94     94
weekday              7      7
month                3      3
hour                 24     24
type                 1      1


In [67]:
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

In [68]:
# As we see, different levels problem is solved.

In [69]:
# Rare Encoding

#mask_train = df.copy().isnull() # Create a mask to have nan values back after encoding
rare_df = rare_encoder(combined_df, 0.0001)

In [70]:
rare_train = rare_df[rare_df['type']=='train']
rare_test = rare_df[rare_df['type']=='test']
del rare_train['type']
del rare_test['type']

In [71]:
# Check if the categories are same for train and test, after rare encoding
cat_cols = ['product_name','brand_name','businessunit','category_id','Level1_Category_Id','Level2_Category_Id']

for col in cat_cols:
    a = set(rare_train[col].dropna())
    b = set(rare_test[col].dropna())
    print(a==b)

True
True
True
True
True
True


In [72]:
# We encoded the rare values as 'Rare' and train and test sets have the same levels. We are ready to go.

In [73]:
#df['sellingprice']=sel_price_tr
#df_test['sellingprice'] = sel_price_te
rare_train.head(1)

,user_action,product_name,brand_name,businessunit,product_gender,category_id,Level1_Category_Id,Level2_Category_Id,weekday,month,hour
0,favorite,Rare,Rare,Beyaz Eşya,Unisex,1272.0,1071.0,1212.0,2,12,22


In [74]:
rare_train.isnull().any()

user_action           False
product_name           True
brand_name             True
businessunit           True
product_gender         True
category_id            True
Level1_Category_Id     True
Level2_Category_Id     True
weekday               False
month                 False
hour                  False
dtype: bool

In [75]:
a = rare_train['Level1_Category_Id']

In [76]:
mask_train = rare_train.copy().isnull() # Create a mask to have nan values back after encoding
original_train = rare_train.copy()
rare_train.fillna("kalamar", inplace=True) # To deal with a issue due to nan-str not being comparable

mask_test = rare_test.copy().isnull() # Create a mask to have nan values back after encoding
original_test = rare_test.copy()
rare_test.fillna("kalamar", inplace=True) # To deal with a issue due to nan-str not being comparable

cols = ['user_action','weekday','month','hour','product_name','brand_name','businessunit','product_gender',
        'category_id' ,'Level1_Category_Id','Level2_Category_Id']
rare_train[cols] = rare_train[cols].astype('str')
rare_test[cols] = rare_test[cols].astype('str')


encoders = {}

x = 0

for i in rare_train:
    if i!="sellingprice":
        # print(i) #Debugging purposes
        le = LabelEncoder()
        le.fit(rare_train[i])
        encoders[x] = le
        x+=1
        
x = 0
train_data = pd.DataFrame()

for i in rare_train:
    # print(i) # Debugging
    if i!="sellingprice":
        le = encoders[x]
        train_data[i] = le.transform(rare_train[i])
        x+=1


x = 0
test_data = pd.DataFrame()
for i in rare_test:
    #print(i) # Debugging
    if i!="sellingprice":
        le = encoders[x]
        test_data[i] = le.transform(rare_test[i])
        x+=1


/opt/anaconda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/opt/anaconda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [77]:
train_data.index = rare_train.index
test_data.index = rare_test.index

final_train = train_data.where(~mask_train, original_train)
final_test = test_data.where(~mask_test, original_test)

In [78]:
# CATBoost

from catboost import CatBoostClassifier, Pool
sell_tr = pd.DataFrame(sel_price_tr, columns=['sellingprice'])
sell_te = pd.DataFrame(sel_price_te, columns=['sellingprice'])

In [79]:
final_final_train = pd.concat([final_train, sel_price_tr], axis=1)
final_final_test = pd.concat([final_test, sel_price_te], axis=1)

In [80]:
cols = ['user_action','weekday','month','hour','product_name','brand_name','businessunit','product_gender',
        'category_id' ,'Level1_Category_Id','Level2_Category_Id']
final_final_train[cols] = final_final_train[cols].astype('str')
final_final_test[cols] = final_final_test[cols].astype('str')

In [81]:
lee = LabelEncoder()
encoded_labels = le.fit_transform(df_gender)

In [82]:
# df_gender
# sell_price_tr
# sell_price_te

train_data = final_final_train
train_label = encoded_labels

train_pool = Pool(train_data, 
                  train_label, 
                  cat_features=cols)
######

test_pool = Pool(final_final_test, 
                  cat_features=cols)


In [83]:
model = CatBoostClassifier(iterations=100, depth=10, learning_rate = 0.01, loss_function='Logloss', verbose=True)

In [85]:
model.fit(train_pool)

0:	learn: 0.6875768	total: 283ms	remaining: 28s
1:	learn: 0.6821242	total: 434ms	remaining: 21.3s
2:	learn: 0.6767727	total: 543ms	remaining: 17.6s
3:	learn: 0.6715391	total: 669ms	remaining: 16s
4:	learn: 0.6663987	total: 813ms	remaining: 15.5s
5:	learn: 0.6613653	total: 932ms	remaining: 14.6s
6:	learn: 0.6564344	total: 1.02s	remaining: 13.5s
7:	learn: 0.6515798	total: 1.15s	remaining: 13.2s
8:	learn: 0.6468143	total: 1.25s	remaining: 12.7s
9:	learn: 0.6421528	total: 1.35s	remaining: 12.2s
10:	learn: 0.6375770	total: 1.47s	remaining: 11.9s
11:	learn: 0.6330797	total: 1.58s	remaining: 11.6s
12:	learn: 0.6286853	total: 1.65s	remaining: 11.1s
13:	learn: 0.6243525	total: 1.78s	remaining: 11s
14:	learn: 0.6201122	total: 1.87s	remaining: 10.6s
15:	learn: 0.6159462	total: 2.04s	remaining: 10.7s
16:	learn: 0.6118568	total: 2.19s	remaining: 10.7s
17:	learn: 0.6078297	total: 2.36s	remaining: 10.7s
18:	learn: 0.6038727	total: 2.54s	remaining: 10.8s
19:	learn: 0.5999803	total: 2.69s	remaining: 10

In [86]:
preds = model.predict(test_pool, 'Probability')

In [103]:
expected_id = pd.read_csv('test_ids_in_prediction.csv')

In [106]:
expected_id.head(2)

,unique_id
0,9
1,18


In [136]:
pred_0 = preds[:,0]
pred_1 = preds[:,1]

results_0 = pd.DataFrame({"id":test_ids, "pred":pred_0})
results_1 = pd.DataFrame({"id":test_ids, "pred":pred_1})

In [137]:
array_0 = results_0.groupby('id')['pred'].mean().values
array_1 = results_1.groupby('id')['pred'].mean().values

In [141]:
array_0 = array_0.round(6)
array_1 = array_1.round(6)

In [142]:
array_0 = array_0.astype(str)
array_1 = array_1.astype(str)

In [143]:
final_0 = ", ".join(list(array_0))
final_1 = ", ".join(list(array_1))

In [144]:
with open('0_prob.txt','w') as f:
    f.write(final_0)

In [145]:
with open('1_prob.txt','w') as f:
    f.write(final_1)

In [147]:
final_0

'0.751511, 0.746846, 0.73491, 0.746456, 0.739169, 0.7675, 0.605904, 0.55, 0.73214, 0.748064, 0.577366, 0.502011, 0.737375, 0.760525, 0.737708, 0.736406, 0.605353, 0.516383, 0.744157, 0.732792, 0.774403, 0.712767, 0.728033, 0.724105, 0.749875, 0.756892, 0.767827, 0.760573, 0.751211, 0.715273, 0.755078, 0.721801, 0.766373, 0.722631, 0.766602, 0.74572, 0.653608, 0.703407, 0.759689, 0.729861, 0.726699, 0.716429, 0.758733, 0.665032, 0.747346, 0.729757, 0.74847, 0.674198, 0.757305, 0.642178, 0.554499, 0.767283, 0.679769, 0.509391, 0.636671, 0.560768, 0.712026, 0.724857, 0.728907, 0.74496, 0.716123, 0.64739, 0.487447, 0.707917, 0.765168, 0.705354, 0.757583, 0.690221, 0.69682, 0.729331, 0.755661, 0.755539, 0.737212, 0.76201, 0.714812, 0.724572, 0.674984, 0.743789, 0.771469, 0.746814, 0.634926, 0.704094, 0.697492, 0.561519, 0.753924, 0.749183, 0.699043, 0.759036, 0.753708, 0.713567, 0.736025, 0.719049, 0.707768, 0.741765, 0.654969, 0.748607, 0.752317, 0.721491, 0.605898, 0.730475, 0.705995, 0.7

# FILL WITH THE MOST FREQUENT

In [51]:
# FOR LIGHTGBM, WE DECIDED NOT TO IMPUTE THE MISSING VALUES.

fill_values = df.mode().iloc[0]
fill_values

user_action                                         visit
sellingprice                                          0.0
product_name          W - Lab Madeleb 40 Ml 8680419062398
brand_name                                  TRENDYOLMİLLA
businessunit                                      Kadın A
product_gender                                      Kadın
category_id                                         601.0
Level1_Category_Id                                  522.0
Level2_Category_Id                                 2871.0
gender                                                  F
unique_id                                            19.0
timestamp_dt                   2020-11-13 18:21:20.017000
weekday                                                 2
month                                                  11
hour                                                   21
Name: 0, dtype: object

In [52]:
df = df.fillna(fill_values)
df_test = df_test.fillna(fill_values)

In [53]:
df.dtypes

user_action                   object
sellingprice                 float64
product_name                  object
brand_name                    object
businessunit                  object
product_gender                object
category_id                  float64
Level1_Category_Id           float64
Level2_Category_Id           float64
gender                        object
unique_id                      int64
timestamp_dt          datetime64[ns]
weekday                        int64
month                          int64
hour                           int64
dtype: object

In [65]:
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

# Rare Encoding

rare_df = rare_encoder(df, 0.001)
rare_test = rare_encoder(df_test, 0.001)

In [66]:
# Check if the categories are same for train and test, after rare encoding

cat_cols = ['product_name','brand_name','businessunit','category_id','Level1_Category_Id','Level2_Category_Id']

for col in cat_cols:
    a = set(rare_df[col].unique())
    b = set(rare_test[col].unique())
    print(a==b)

True
False
True
False
True
False


# RF IMPUTING STRATEGY

In [76]:
columns = ['product_name', 'brand_name', 'businessunit', 'category_id', 'Level1_Category_Id', 'Level2_Category_Id']

random_dict = {}

for column in columns:
    s = df[column].notnull()
    null = s[~s].index
    not_null = s[s].index
    
    df_notnull = df.loc[not_null, :]
    df_notnull_x = df_notnull[['weekday','month','hour']]  #'user_action', doesn't work with RF. 
    df_notnull_y = df_notnull[column]
    
    rf_impute = RandomForestClassifier(n_estimators = 100)
    rf_impute.fit(df_notnull_x, df_notnull_y)
    
    df_null = df.loc[null, :]
    df_null_x = df_null[['weekday','month','hour']] #'user_action', doesn't work with RF.
    df_null_y = rf_impute.predict(df_null_x)
    
    random_dict[f'{column}'] = df_null_y
    break
    
# could not allocate 1808261120 bytes

MemoryError: could not allocate 1808261120 bytes